In [1]:
#配置环境
import urllib.request
import requests
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd

headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

In [2]:
# 导入写入的广州二手房数据
df = pd.read_csv(r'./小区网址.csv')
df

,Unnamed: 0,id,num
0,0,c2110111597161952/,114
1,1,c2110115377016126/,80
2,2,c2110115187486483/,203
3,3,c2110343238336996/,53
4,4,c2110343238860955/,91
...,...,...,...
2958,2993,c2111103318070/,3
2959,2994,c2111856330108176/,0
2960,2995,c2120020596176813/,0
2961,2996,c2110111331611194/,0


In [3]:
#查看小区网址信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963 entries, 0 to 2962
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2963 non-null   int64 
 1   id          2963 non-null   object
 2   num         2963 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 69.6+ KB


In [4]:
#查看小区网址中没有二手房源的网址，并剔除
df = df[df['num'] > 0]
df

,Unnamed: 0,id,num
0,0,c2110111597161952/,114
1,1,c2110115377016126/,80
2,2,c2110115187486483/,203
3,3,c2110343238336996/,53
4,4,c2110343238860955/,91
...,...,...,...
2953,2988,c2111103316927/,7
2956,2991,c2111103317892/,1
2957,2992,c2111103318560/,11
2958,2993,c2111103318070/,3


In [5]:
#查看df详细信息
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2601 entries, 0 to 2962
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2601 non-null   int64 
 1   id          2601 non-null   object
 2   num         2601 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 81.3+ KB


In [6]:
#重置引索
df.index = range(len(df))
df

,Unnamed: 0,id,num
0,0,c2110111597161952/,114
1,1,c2110115377016126/,80
2,2,c2110115187486483/,203
3,3,c2110343238336996/,53
4,4,c2110343238860955/,91
...,...,...,...
2596,2988,c2111103316927/,7
2597,2991,c2111103317892/,1
2598,2992,c2111103318560/,11
2599,2993,c2111103318070/,3


In [7]:
#删除多余列
df = df.drop('Unnamed: 0', axis = 1)
df

,id,num
0,c2110111597161952/,114
1,c2110115377016126/,80
2,c2110115187486483/,203
3,c2110343238336996/,53
4,c2110343238860955/,91
...,...,...
2596,c2111103316927/,7
2597,c2111103317892/,1
2598,c2111103318560/,11
2599,c2111103318070/,3


In [8]:
HouseName_list = []         #楼盘名列表
address_list = []           #楼盘地址列表
houseInfo_lise = []         #楼盘详细信息列表
Type_list = []              #户型列表
Area_list = []              #建筑面积列表
Direction_list = []         #朝向列表
Dectype_list = []           #装修类型列表
Floor_list = []             #楼层列表
year_list = []              #年份列表
banta_list = []             #板塔列表
totalPrice_list = []        #楼盘总价格列表
unitPrice_list = []         #楼盘单价（元/平方米）列表
region_list = []            #楼盘区域（区）

def region_xq(x):
    url = 'https://gz.lianjia.com/ershoufang/pg1' + df['id'][x]
    response = requests.get(url, headers = headers)
    data = response.text
    soup = BeautifulSoup(data, "html5lib")
    #获取区域
    #需要先进入详情页
    region_web = soup.select('#content > div.leftContent > ul > li:nth-child(1) > div.info.clear > div.title > a')
    #爬取详情页网页的<a href>
    region_web = region_web[0]['href']
    #创建一个url加载网页
    region_url = region_web
    res = requests.get(region_url, headers = headers)
    region_data = res.text
    region_soup = BeautifulSoup(region_data, "html5lib")
    #爬取房源区域
    region = region_soup.select('body > div.overview > div.content > div.aroundInfo > div.areaName > span.info > a:nth-child(1)')
    region = region[0].get_text()
#     region_list.append(region)
#       print(region)
    return region




#通过爬取每个id获取该小区房源

def house(x):
    try:
        for z in range(x, len(df)):     
            #如果数量大于每页所取的30个房源信息，需要查询需要多少页，最后一页有多少信息。
            if df['num'][z] > 30:
                if df['num'][z] % 30 == 0:
                    page = df['num'][z] // 30
                else:
                    page = df['num'][z] // 30 + 1
            else:
                page = 1
            print('第'+str(z+1)+'个小区！需要爬取信息'+str(df['num'][z])+'条！')
            
            #每个小区对应一个区域，所以可以提取出来，避免重复爬取。
            
            url = 'https://gz.lianjia.com/ershoufang/pg1' + df['id'][x]
            response = requests.get(url, headers = headers)
            data = response.text
            soup = BeautifulSoup(data, "html5lib")
            #获取区域
            #需要先进入详情页
            region_web = soup.select('#content > div.leftContent > ul > li:nth-child(1) > div.info.clear > div.title > a')
            #爬取详情页网页的<a href>
            region_web = region_web[0]['href']
            #创建一个url加载网页
            region_url = region_web
            res = requests.get(region_url, headers = headers)
            region_data = res.text
            region_soup = BeautifulSoup(region_data, "html5lib")
            #爬取房源区域
            region = region_soup.select('body > div.overview > div.content > div.aroundInfo > div.areaName > span.info > a:nth-child(1)')
            region = region[0].get_text()
        #     region_list.append(region)
        #       print(region)
            
            for q in range(0, page):
                url = 'https://gz.lianjia.com/ershoufang/pg' + str(q+1) + df['id'][z]
                response = requests.get(url, headers = headers)
                data = response.text
                soup = BeautifulSoup(data, "html5lib")
                if df['num'][z] > 30:
                    if df['num'][z] % 30 == 0:
                        last_page_num = 30
                    else:
                        page_num = page - (q+1)
                        if page_num == 0:
                            last_page_num = df['num'][z] % 30
                        else:
                            last_page_num = 30
                else:
                    last_page_num = df['num'][z]
                    
#                 print('第'+str(q+1)+'页！需要爬取信息'+str(last_page_num)+'条！')
                for i in range(0, last_page_num):

                    #获取楼盘名信息
                    HouseName = soup.select("#content > div.leftContent > ul > li:nth-child(" + str(i+1) + ") > div.info.clear > div.flood > div")
                    HouseName = HouseName[0].get_text().replace(" ","").split('-')[0]
                    HouseName_list.append(HouseName)
#                     print(HouseName)

                    #获取楼盘地址
                    address = soup.select("#content > div.leftContent > ul > li:nth-child(" + str(i+1) + ") > div.info.clear > div.flood > div")
                    address = address[0].get_text().replace(" ","").split('-')[1]
                    address_list.append(address)
#                     print(address)

                    #获取楼盘具体信息
                    houseInfo = soup.select("#content > div.leftContent > ul > li:nth-child(" + str(i+1) + ") > div.info.clear > div.address > div")
                    houseInfo = houseInfo[0].get_text()
                    houseInfo_lise.append(houseInfo)
#                     print(houseInfo)

                    #获取楼盘总价格
                    totalPrice = soup.select('#content > div.leftContent > ul > li:nth-child(' + str(i+1) +') > div.info.clear > div.priceInfo > div.totalPrice')
                    totalPrice = totalPrice[0].get_text().replace('万','')
                    totalPrice_list.append(totalPrice)
#                     print(totalPrice)

                    #获取楼盘单价（元/平方米）
                    unitPrice = soup.select('#content > div.leftContent > ul > li:nth-child(' + str(i+1) + ') > div.info.clear > div.priceInfo > div.unitPrice > span')
                    unitPrice = unitPrice[0].get_text().replace('单价','').replace('元/平米','')
                    unitPrice_list.append(unitPrice)
#                     print(unitPrice)
                    
                    #获取区域
                    region_list.append(region)
#                     print(region)
            print('第'+str(z+1)+'个爬取完成！爬取信息'+str(df['num'][z])+'条！')
    except Exception as e:
        #本来可以不用try的，但是调试中发现爬一半的时候有的房子被卖了。。造成错误。
        #房源数量未更新，手动减一
        df.loc[z, 'num'] = df['num'][z] - 1
        print('小区房源有更新，需要重新爬取，正在重新爬取中...')
        if df['num'][z] == 0:
            print('网址被抓，已保存到第'+str(z)+'页！请更新页码重新爬取！')
        else:
            return house(z)
#         print(e)


In [9]:
#运行程序,从第一条开始爬起
house(0)

第1个小区！需要爬取信息114条！
第1个爬取完成！爬取信息114条！
第2个小区！需要爬取信息80条！
第2个爬取完成！爬取信息80条！
第3个小区！需要爬取信息203条！
第3个爬取完成！爬取信息203条！
第4个小区！需要爬取信息53条！
第4个爬取完成！爬取信息53条！
第5个小区！需要爬取信息91条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第5个小区！需要爬取信息90条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第5个小区！需要爬取信息89条！
第5个爬取完成！爬取信息89条！
第6个小区！需要爬取信息114条！
第6个爬取完成！爬取信息114条！
第7个小区！需要爬取信息59条！
第7个爬取完成！爬取信息59条！
第8个小区！需要爬取信息118条！
第8个爬取完成！爬取信息118条！
第9个小区！需要爬取信息133条！
第9个爬取完成！爬取信息133条！
第10个小区！需要爬取信息32条！
第10个爬取完成！爬取信息32条！
第11个小区！需要爬取信息106条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第11个小区！需要爬取信息105条！
第11个爬取完成！爬取信息105条！
第12个小区！需要爬取信息106条！
第12个爬取完成！爬取信息106条！
第13个小区！需要爬取信息81条！
第13个爬取完成！爬取信息81条！
第14个小区！需要爬取信息22条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第14个小区！需要爬取信息21条！
第14个爬取完成！爬取信息21条！
第15个小区！需要爬取信息105条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第15个小区！需要爬取信息104条！
第15个爬取完成！爬取信息104条！
第16个小区！需要爬取信息277条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第16个小区！需要爬取信息276条！
第16个爬取完成！爬取信息276条！
第17个小区！需要爬取信息63条！
第17个爬取完成！爬取信息63条！
第18个小区！需要爬取信息261条！
第18个爬取完成！爬取信息261条！
第19个小区！需要爬取信息265条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第19个小区！需要爬取信息264条！
第19个爬取完成！爬取信息264条！
第2

小区房源有更新，需要重新爬取，正在重新爬取中...
第172个小区！需要爬取信息62条！
第172个爬取完成！爬取信息62条！
第173个小区！需要爬取信息28条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第173个小区！需要爬取信息27条！
第173个爬取完成！爬取信息27条！
第174个小区！需要爬取信息1条！
第174个爬取完成！爬取信息1条！
第175个小区！需要爬取信息26条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第175个小区！需要爬取信息25条！
第175个爬取完成！爬取信息25条！
第176个小区！需要爬取信息10条！
第176个爬取完成！爬取信息10条！
第177个小区！需要爬取信息26条！
第177个爬取完成！爬取信息26条！
第178个小区！需要爬取信息76条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第178个小区！需要爬取信息75条！
第178个爬取完成！爬取信息75条！
第179个小区！需要爬取信息27条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第179个小区！需要爬取信息26条！
第179个爬取完成！爬取信息26条！
第180个小区！需要爬取信息4条！
第180个爬取完成！爬取信息4条！
第181个小区！需要爬取信息15条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第181个小区！需要爬取信息14条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第181个小区！需要爬取信息13条！
第181个爬取完成！爬取信息13条！
第182个小区！需要爬取信息36条！
第182个爬取完成！爬取信息36条！
第183个小区！需要爬取信息23条！
第183个爬取完成！爬取信息23条！
第184个小区！需要爬取信息155条！
第184个爬取完成！爬取信息155条！
第185个小区！需要爬取信息87条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第185个小区！需要爬取信息86条！
第185个爬取完成！爬取信息86条！
第186个小区！需要爬取信息42条！
第186个爬取完成！爬取信息42条！
第187个小区！需要爬取信息78条！
第187个爬取完成！爬取信息78条！
第188个小区！需要爬取信息48条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第188个小区！

第341个爬取完成！爬取信息39条！
第342个小区！需要爬取信息25条！
第342个爬取完成！爬取信息25条！
第343个小区！需要爬取信息13条！
第343个爬取完成！爬取信息13条！
第344个小区！需要爬取信息8条！
第344个爬取完成！爬取信息8条！
第345个小区！需要爬取信息12条！
第345个爬取完成！爬取信息12条！
第346个小区！需要爬取信息11条！
第346个爬取完成！爬取信息11条！
第347个小区！需要爬取信息47条！
第347个爬取完成！爬取信息47条！
第348个小区！需要爬取信息104条！
第348个爬取完成！爬取信息104条！
第349个小区！需要爬取信息200条！
第349个爬取完成！爬取信息200条！
第350个小区！需要爬取信息15条！
第350个爬取完成！爬取信息15条！
第351个小区！需要爬取信息44条！
第351个爬取完成！爬取信息44条！
第352个小区！需要爬取信息5条！
第352个爬取完成！爬取信息5条！
第353个小区！需要爬取信息41条！
第353个爬取完成！爬取信息41条！
第354个小区！需要爬取信息83条！
第354个爬取完成！爬取信息83条！
第355个小区！需要爬取信息91条！
第355个爬取完成！爬取信息91条！
第356个小区！需要爬取信息108条！
第356个爬取完成！爬取信息108条！
第357个小区！需要爬取信息15条！
第357个爬取完成！爬取信息15条！
第358个小区！需要爬取信息11条！
第358个爬取完成！爬取信息11条！
第359个小区！需要爬取信息4条！
第359个爬取完成！爬取信息4条！
第360个小区！需要爬取信息20条！
第360个爬取完成！爬取信息20条！
第361个小区！需要爬取信息15条！
第361个爬取完成！爬取信息15条！
第362个小区！需要爬取信息67条！
第362个爬取完成！爬取信息67条！
第363个小区！需要爬取信息36条！
第363个爬取完成！爬取信息36条！
第364个小区！需要爬取信息3条！
第364个爬取完成！爬取信息3条！
第365个小区！需要爬取信息7条！
第365个爬取完成！爬取信息7条！
第366个小区！需要爬取信息5条！
第366个爬取完成！爬取信息5条！
第367个小区！需要爬取信息20条！
第367个爬取完成！爬取信息20条！

第529个爬取完成！爬取信息74条！
第530个小区！需要爬取信息44条！
第530个爬取完成！爬取信息44条！
第531个小区！需要爬取信息2条！
第531个爬取完成！爬取信息2条！
第532个小区！需要爬取信息14条！
第532个爬取完成！爬取信息14条！
第533个小区！需要爬取信息25条！
第533个爬取完成！爬取信息25条！
第534个小区！需要爬取信息6条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第534个小区！需要爬取信息5条！
第534个爬取完成！爬取信息5条！
第535个小区！需要爬取信息62条！
第535个爬取完成！爬取信息62条！
第536个小区！需要爬取信息85条！
第536个爬取完成！爬取信息85条！
第537个小区！需要爬取信息32条！
第537个爬取完成！爬取信息32条！
第538个小区！需要爬取信息63条！
第538个爬取完成！爬取信息63条！
第539个小区！需要爬取信息3条！
第539个爬取完成！爬取信息3条！
第540个小区！需要爬取信息41条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第540个小区！需要爬取信息40条！
第540个爬取完成！爬取信息40条！
第541个小区！需要爬取信息41条！
第541个爬取完成！爬取信息41条！
第542个小区！需要爬取信息12条！
第542个爬取完成！爬取信息12条！
第543个小区！需要爬取信息167条！
第543个爬取完成！爬取信息167条！
第544个小区！需要爬取信息17条！
第544个爬取完成！爬取信息17条！
第545个小区！需要爬取信息28条！
第545个爬取完成！爬取信息28条！
第546个小区！需要爬取信息8条！
第546个爬取完成！爬取信息8条！
第547个小区！需要爬取信息27条！
第547个爬取完成！爬取信息27条！
第548个小区！需要爬取信息26条！
第548个爬取完成！爬取信息26条！
第549个小区！需要爬取信息23条！
第549个爬取完成！爬取信息23条！
第550个小区！需要爬取信息7条！
第550个爬取完成！爬取信息7条！
第551个小区！需要爬取信息15条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第551个小区！需要爬取信息14条！
第551个爬取完成！爬取信息14条！
第552个小区！需要爬取信息7条！
第

第718个爬取完成！爬取信息4条！
第719个小区！需要爬取信息14条！
第719个爬取完成！爬取信息14条！
第720个小区！需要爬取信息72条！
第720个爬取完成！爬取信息72条！
第721个小区！需要爬取信息31条！
第721个爬取完成！爬取信息31条！
第722个小区！需要爬取信息15条！
第722个爬取完成！爬取信息15条！
第723个小区！需要爬取信息42条！
第723个爬取完成！爬取信息42条！
第724个小区！需要爬取信息7条！
第724个爬取完成！爬取信息7条！
第725个小区！需要爬取信息35条！
第725个爬取完成！爬取信息35条！
第726个小区！需要爬取信息8条！
第726个爬取完成！爬取信息8条！
第727个小区！需要爬取信息60条！
第727个爬取完成！爬取信息60条！
第728个小区！需要爬取信息12条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第728个小区！需要爬取信息11条！
第728个爬取完成！爬取信息11条！
第729个小区！需要爬取信息2条！
第729个爬取完成！爬取信息2条！
第730个小区！需要爬取信息5条！
第730个爬取完成！爬取信息5条！
第731个小区！需要爬取信息29条！
第731个爬取完成！爬取信息29条！
第732个小区！需要爬取信息28条！
第732个爬取完成！爬取信息28条！
第733个小区！需要爬取信息24条！
第733个爬取完成！爬取信息24条！
第734个小区！需要爬取信息21条！
第734个爬取完成！爬取信息21条！
第735个小区！需要爬取信息22条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第735个小区！需要爬取信息21条！
第735个爬取完成！爬取信息21条！
第736个小区！需要爬取信息32条！
第736个爬取完成！爬取信息32条！
第737个小区！需要爬取信息57条！
第737个爬取完成！爬取信息57条！
第738个小区！需要爬取信息14条！
第738个爬取完成！爬取信息14条！
第739个小区！需要爬取信息6条！
第739个爬取完成！爬取信息6条！
第740个小区！需要爬取信息22条！
第740个爬取完成！爬取信息22条！
第741个小区！需要爬取信息4条！
第741个爬取完成！爬取信息4条！
第742个小区！需要爬取信息4条！
第742个爬取完成！爬取

...

第1680个爬取完成！爬取信息3条！
第1681个小区！需要爬取信息2条！
第1681个爬取完成！爬取信息2条！
第1682个小区！需要爬取信息24条！
第1682个爬取完成！爬取信息24条！
第1683个小区！需要爬取信息13条！
第1683个爬取完成！爬取信息13条！
第1684个小区！需要爬取信息14条！
第1684个爬取完成！爬取信息14条！
第1685个小区！需要爬取信息3条！
第1685个爬取完成！爬取信息3条！
第1686个小区！需要爬取信息14条！
第1686个爬取完成！爬取信息14条！
第1687个小区！需要爬取信息10条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第1687个小区！需要爬取信息9条！
第1687个爬取完成！爬取信息9条！
第1688个小区！需要爬取信息12条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第1688个小区！需要爬取信息11条！
第1688个爬取完成！爬取信息11条！
第1689个小区！需要爬取信息3条！
第1689个爬取完成！爬取信息3条！
第1690个小区！需要爬取信息12条！
第1690个爬取完成！爬取信息12条！
第1691个小区！需要爬取信息36条！
第1691个爬取完成！爬取信息36条！
第1692个小区！需要爬取信息6条！
第1692个爬取完成！爬取信息6条！
第1693个小区！需要爬取信息1条！
第1693个爬取完成！爬取信息1条！
第1694个小区！需要爬取信息2条！
第1694个爬取完成！爬取信息2条！
第1695个小区！需要爬取信息4条！
第1695个爬取完成！爬取信息4条！
第1696个小区！需要爬取信息2条！
第1696个爬取完成！爬取信息2条！
第1697个小区！需要爬取信息1条！
第1697个爬取完成！爬取信息1条！
第1698个小区！需要爬取信息4条！
第1698个爬取完成！爬取信息4条！
第1699个小区！需要爬取信息65条！
第1699个爬取完成！爬取信息65条！
第1700个小区！需要爬取信息43条！
第1700个爬取完成！爬取信息43条！
第1701个小区！需要爬取信息11条！
第1701个爬取完成！爬取信息11条！
第1702个小区！需要爬取信息4条！
第1702个爬取完成！爬取信息4条！
第1703个小区！需要爬取信息25条！
第1703个爬取完成！爬取

第1867个爬取完成！爬取信息5条！
第1868个小区！需要爬取信息4条！
第1868个爬取完成！爬取信息4条！
第1869个小区！需要爬取信息17条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第1869个小区！需要爬取信息16条！
第1869个爬取完成！爬取信息16条！
第1870个小区！需要爬取信息9条！
第1870个爬取完成！爬取信息9条！
第1871个小区！需要爬取信息2条！
第1871个爬取完成！爬取信息2条！
第1872个小区！需要爬取信息9条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第1872个小区！需要爬取信息8条！
第1872个爬取完成！爬取信息8条！
第1873个小区！需要爬取信息52条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第1873个小区！需要爬取信息51条！
第1873个爬取完成！爬取信息51条！
第1874个小区！需要爬取信息9条！
第1874个爬取完成！爬取信息9条！
第1875个小区！需要爬取信息5条！
第1875个爬取完成！爬取信息5条！
第1876个小区！需要爬取信息4条！
第1876个爬取完成！爬取信息4条！
第1877个小区！需要爬取信息8条！
第1877个爬取完成！爬取信息8条！
第1878个小区！需要爬取信息4条！
第1878个爬取完成！爬取信息4条！
第1879个小区！需要爬取信息1条！
第1879个爬取完成！爬取信息1条！
第1880个小区！需要爬取信息11条！
第1880个爬取完成！爬取信息11条！
第1881个小区！需要爬取信息45条！
第1881个爬取完成！爬取信息45条！
第1882个小区！需要爬取信息6条！
第1882个爬取完成！爬取信息6条！
第1883个小区！需要爬取信息13条！
第1883个爬取完成！爬取信息13条！
第1884个小区！需要爬取信息5条！
第1884个爬取完成！爬取信息5条！
第1885个小区！需要爬取信息3条！
第1885个爬取完成！爬取信息3条！
第1886个小区！需要爬取信息54条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第1886个小区！需要爬取信息53条！
第1886个爬取完成！爬取信息53条！
第1887个小区！需要爬取信息5条！
第1887个爬取完成！爬取信息5条！
第1888个小区！需要爬取信息17条！
第1888个

第2065个爬取完成！爬取信息2条！
第2066个小区！需要爬取信息4条！
第2066个爬取完成！爬取信息4条！
第2067个小区！需要爬取信息7条！
第2067个爬取完成！爬取信息7条！
第2068个小区！需要爬取信息50条！
第2068个爬取完成！爬取信息50条！
第2069个小区！需要爬取信息19条！
第2069个爬取完成！爬取信息19条！
第2070个小区！需要爬取信息6条！
第2070个爬取完成！爬取信息6条！
第2071个小区！需要爬取信息8条！
第2071个爬取完成！爬取信息8条！
第2072个小区！需要爬取信息8条！
第2072个爬取完成！爬取信息8条！
第2073个小区！需要爬取信息14条！
第2073个爬取完成！爬取信息14条！
第2074个小区！需要爬取信息18条！
第2074个爬取完成！爬取信息18条！
第2075个小区！需要爬取信息1条！
第2075个爬取完成！爬取信息1条！
第2076个小区！需要爬取信息28条！
第2076个爬取完成！爬取信息28条！
第2077个小区！需要爬取信息13条！
第2077个爬取完成！爬取信息13条！
第2078个小区！需要爬取信息5条！
第2078个爬取完成！爬取信息5条！
第2079个小区！需要爬取信息43条！
第2079个爬取完成！爬取信息43条！
第2080个小区！需要爬取信息15条！
第2080个爬取完成！爬取信息15条！
第2081个小区！需要爬取信息1条！
第2081个爬取完成！爬取信息1条！
第2082个小区！需要爬取信息1条！
第2082个爬取完成！爬取信息1条！
第2083个小区！需要爬取信息6条！
第2083个爬取完成！爬取信息6条！
第2084个小区！需要爬取信息13条！
第2084个爬取完成！爬取信息13条！
第2085个小区！需要爬取信息7条！
第2085个爬取完成！爬取信息7条！
第2086个小区！需要爬取信息3条！
第2086个爬取完成！爬取信息3条！
第2087个小区！需要爬取信息5条！
第2087个爬取完成！爬取信息5条！
第2088个小区！需要爬取信息18条！
第2088个爬取完成！爬取信息18条！
第2089个小区！需要爬取信息7条！
第2089个爬取完成！爬取信息7条！
第2090个小区！需要爬取信息18条！
第2090个爬取完成！爬取信息18条！
第2091个小区！

第2262个爬取完成！爬取信息3条！
第2263个小区！需要爬取信息1条！
第2263个爬取完成！爬取信息1条！
第2264个小区！需要爬取信息20条！
第2264个爬取完成！爬取信息20条！
第2265个小区！需要爬取信息1条！
第2265个爬取完成！爬取信息1条！
第2266个小区！需要爬取信息6条！
第2266个爬取完成！爬取信息6条！
第2267个小区！需要爬取信息6条！
第2267个爬取完成！爬取信息6条！
第2268个小区！需要爬取信息19条！
第2268个爬取完成！爬取信息19条！
第2269个小区！需要爬取信息11条！
第2269个爬取完成！爬取信息11条！
第2270个小区！需要爬取信息12条！
第2270个爬取完成！爬取信息12条！
第2271个小区！需要爬取信息1条！
第2271个爬取完成！爬取信息1条！
第2272个小区！需要爬取信息10条！
第2272个爬取完成！爬取信息10条！
第2273个小区！需要爬取信息4条！
第2273个爬取完成！爬取信息4条！
第2274个小区！需要爬取信息9条！
第2274个爬取完成！爬取信息9条！
第2275个小区！需要爬取信息3条！
第2275个爬取完成！爬取信息3条！
第2276个小区！需要爬取信息1条！
第2276个爬取完成！爬取信息1条！
第2277个小区！需要爬取信息3条！
第2277个爬取完成！爬取信息3条！
第2278个小区！需要爬取信息6条！
第2278个爬取完成！爬取信息6条！
第2279个小区！需要爬取信息8条！
第2279个爬取完成！爬取信息8条！
第2280个小区！需要爬取信息2条！
第2280个爬取完成！爬取信息2条！
第2281个小区！需要爬取信息4条！
第2281个爬取完成！爬取信息4条！
第2282个小区！需要爬取信息2条！
第2282个爬取完成！爬取信息2条！
第2283个小区！需要爬取信息13条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第2283个小区！需要爬取信息12条！
第2283个爬取完成！爬取信息12条！
第2284个小区！需要爬取信息4条！
第2284个爬取完成！爬取信息4条！
第2285个小区！需要爬取信息7条！
第2285个爬取完成！爬取信息7条！
第2286个小区！需要爬取信息1条！
第2286个爬取完成！爬取信息1条！
第2287个小区！需要

第2467个爬取完成！爬取信息1条！
第2468个小区！需要爬取信息1条！
第2468个爬取完成！爬取信息1条！
第2469个小区！需要爬取信息1条！
第2469个爬取完成！爬取信息1条！
第2470个小区！需要爬取信息1条！
第2470个爬取完成！爬取信息1条！
第2471个小区！需要爬取信息1条！
第2471个爬取完成！爬取信息1条！
第2472个小区！需要爬取信息1条！
第2472个爬取完成！爬取信息1条！
第2473个小区！需要爬取信息1条！
第2473个爬取完成！爬取信息1条！
第2474个小区！需要爬取信息1条！
第2474个爬取完成！爬取信息1条！
第2475个小区！需要爬取信息1条！
第2475个爬取完成！爬取信息1条！
第2476个小区！需要爬取信息1条！
第2476个爬取完成！爬取信息1条！
第2477个小区！需要爬取信息1条！
第2477个爬取完成！爬取信息1条！
第2478个小区！需要爬取信息2条！
小区房源有更新，需要重新爬取，正在重新爬取中...
第2478个小区！需要爬取信息1条！
第2478个爬取完成！爬取信息1条！
第2479个小区！需要爬取信息1条！
第2479个爬取完成！爬取信息1条！
第2480个小区！需要爬取信息1条！
第2480个爬取完成！爬取信息1条！
第2481个小区！需要爬取信息1条！
第2481个爬取完成！爬取信息1条！
第2482个小区！需要爬取信息1条！
第2482个爬取完成！爬取信息1条！
第2483个小区！需要爬取信息1条！
第2483个爬取完成！爬取信息1条！
第2484个小区！需要爬取信息1条！
第2484个爬取完成！爬取信息1条！
第2485个小区！需要爬取信息1条！
第2485个爬取完成！爬取信息1条！
第2486个小区！需要爬取信息1条！
第2486个爬取完成！爬取信息1条！
第2487个小区！需要爬取信息1条！
第2487个爬取完成！爬取信息1条！
第2488个小区！需要爬取信息1条！
第2488个爬取完成！爬取信息1条！
第2489个小区！需要爬取信息1条！
第2489个爬取完成！爬取信息1条！
第2490个小区！需要爬取信息1条！
第2490个爬取完成！爬取信息1条！
第2491个小区！需要爬取信息1条！
第2491个爬取完成！爬取信息1条！
第2492个小区！需要爬取信息1条！
第2492

In [11]:
print(len(HouseName_list))
print(len(address_list))
print(len(houseInfo_lise))
print(len(totalPrice_list))
print(len(unitPrice_list))
print(len(region_list))

68625
68625
68625
68625
68625
68625


In [13]:
#将爬取到的信息转换为字典，并用DataFrame输出
from pandas import DataFrame
dict_house = {'HouseName':HouseName_list,'Address':address_list,'houseInfo':houseInfo_lise,'totalPrice':totalPrice_list,'unitPrice':unitPrice_list,'region':region_list}
d_house = DataFrame(dict_house)
d_house

,HouseName,Address,houseInfo,totalPrice,unitPrice,region
0,中海誉城北苑,香雪,1室1厅 | 56.38平米 | 东 | 精装 | 中楼层(共33层) | 2012年建 | 塔楼,223,39554,黄埔
1,中海誉城北苑,香雪,2室2厅 | 87.7平米 | 西南 | 精装 | 低楼层(共33层) | 2012年建 | 塔楼,320,36489,黄埔
2,中海誉城北苑,香雪,1室1厅 | 56.38平米 | 北 | 精装 | 高楼层(共33层) | 2012年建 | 塔楼,212,37602,黄埔
3,中海誉城北苑,香雪,3室2厅 | 97.1平米 | 东南 | 精装 | 中楼层(共33层) | 塔楼,375,38620,黄埔
4,中海誉城北苑,香雪,3室2厅 | 92.45平米 | 东 | 精装 | 高楼层(共33层) | 塔楼,370,40022,黄埔
...,...,...,...,...,...,...
68620,华仙楼,沙太南,3室2厅 | 91.81平米 | 南 北 | 其他 | 高楼层(共9层) | 1999年建 ...,320,34855,越秀
68621,华仙楼,沙太南,1室1厅 | 36.06平米 | 东南 西南 | 精装 | 高楼层(共9层) | 塔楼,115,31892,越秀
68622,田美居,同德围,2室1厅 | 73.4平米 | 南 | 简装 | 高楼层(共9层) | 1999年建 | 塔楼,210,28611,越秀
68623,田美居,同德围,2室1厅 | 73.5平米 | 南 | 简装 | 高楼层(共9层) | 1999年建 | 塔楼,184,25035,越秀


In [14]:
#查看因房源更新，重复爬取的列
d_house.duplicated().sum()

3681

In [15]:
#删除重复列并重新查看信息
d_house = d_house.drop_duplicates()
d_house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64944 entries, 0 to 68624
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   HouseName   64944 non-null  object
 1   Address     64944 non-null  object
 2   houseInfo   64944 non-null  object
 3   totalPrice  64944 non-null  object
 4   unitPrice   64944 non-null  object
 5   region      64944 non-null  object
dtypes: object(6)
memory usage: 3.5+ MB


In [16]:
#将房源信息保存到csv中
import csv
d_house.to_csv('广州二手房源信息.csv')